In [12]:
import os 
import sys
import re 

import pandas as pd 
import numpy as np 
import torch
import torchvision
import cv2 as cv 
import glob
import json 
current_dir = os.getcwd()

 
dataset_dir = os.path.join(current_dir, 'dataset')


sys.path.insert(0, dataset_dir)


### Brainwash Dataset 

In [13]:
def replace_multiple(text, replacements):
    # Create a regex pattern that matches any of the keys
    pattern = re.compile("|".join(re.escape(key) for key in replacements.keys()))
    
    # Function to return the replacement value
    return pattern.sub(lambda match: replacements[match.group(0)], text)


def turn_toInt(x):
    x = x.lstrip().rstrip()    
    x =int(float(x))
    return x


replacements = {
    '(' :"",
    ')' : "",
    ';': "",
}

In [25]:


file_paths = glob.glob(os.path.join(dataset_dir, 'brainwash', '*.idl'), recursive=True)







for file_path in file_paths:
    print(file_path)
    with open(file_path,'r') as file:
        content = file.readlines()

    coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 1,
            "name": "person",  
            "supercategory": "none"
        }
    ]
}




    bb_per_line = []
    for i,line in enumerate(content):
        line_arr  = line.rstrip().split(':')
        if(len(line_arr)==2):
            img_name = line_arr[0]
            boxes =  replace_multiple(line_arr[1],replacements).split(',')
            
            boxes = np.asarray(list(map(turn_toInt,boxes)))
            boxes = boxes.reshape(-1, 4)
            boxes= list(boxes)
            bb_per_line.append(boxes)
            image_info = {
                "id": i + 1,  # Unique image ID
                "file_name": img_name,
                "width": 640,  # Set your image width
                "height": 480  # Set your image height
            }
            coco_format["images"].append(image_info)
            for bbox_id, bbox in enumerate(boxes, start=1):
                x_min, y_min, x_max, y_max = bbox
                width = x_max - x_min
                height = y_max - y_min
                coco_format["annotations"].append({
                    "id": i + 1,
                    "image_id": image_info["id"],
                    "category_id": 1,  # Corresponding category ID
                    "bbox": [int(x_min), int(y_min), int(width), int(height)],  # COCO format: [x, y, width, height]
                    "area": float(width * height),  # Area of the bounding box
                    "iscrowd": 0,  # 0 if not a crowd
                    "segmentation": []
                })
            
    file_name_with_extension = os.path.basename(file_path)
    output_file, _ = os.path.splitext(file_name_with_extension)
    output_path = os.path.join('dataset\\brainwash',f'{output_file}.json')
    with open(output_path, 'w') as json_file:
        
        json.dump(coco_format, json_file, indent=4)

        print(f"COCO format data has been saved to {output_path}.")
        
    

c:\Users\david\Git\YOLOV\preprocessing\dataset\brainwash\brainwash_test.idl
COCO format data has been saved to dataset\brainwash\brainwash_test.json.
c:\Users\david\Git\YOLOV\preprocessing\dataset\brainwash\brainwash_train.idl
COCO format data has been saved to dataset\brainwash\brainwash_train.json.
c:\Users\david\Git\YOLOV\preprocessing\dataset\brainwash\brainwash_val.idl
COCO format data has been saved to dataset\brainwash\brainwash_val.json.


## CamVid

In [34]:
label_mask =[]
label_df = pd.read_csv("dataset\CamVid\class_dict.csv")
label_df





,name,r,g,b
0,Bicyclist,0,128,192
1,Child,192,128,64
2,Pedestrian,64,64,0
